In [1]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.datasets import make_classification
from sklearn.gaussian_process.kernels import RBF

import numpy as np
import matplotlib.pyplot as plt

from cv_svm import SVM_smooth
from kernel_svm import SVM_smooth_kernel
from sampler import sample_from_logreg

from sklearn.preprocessing import normalize

import time

import matplotlib

In [2]:
#matplotlib.use("pgf")
#matplotlib.rcParams.update({
#    "pgf.texsystem": "pdflatex",
#    'font.family': 'serif',
#    'text.usetex': True,
#    'pgf.rcfonts': False,
#})

In [3]:
sigma = 1e-5
lbd = 1
experiment_sizes = np.linspace(1000, 20000, 5)

time_taken = {
    "IACV": np.ones(len(experiment_sizes)),
    "true": np.ones(len(experiment_sizes))
}

err_approx = []
err_cv = []

In [4]:
def Phi_m(v):
    return (1 + v / np.sqrt(1 + v**2)) / 2

def Phi_m_prime(v):
    return 1 / 2 * 1 / (np.sqrt(v**2 + 1) ** 3)

def phi_m(v):
    return 1 / (2 * np.sqrt(1 + v**3))

def Psi_m(alpha, sigma):
    return (
        Phi_m((1 - alpha) / sigma) * (1 - alpha)
        + phi_m((1 - alpha) / sigma) * sigma
    )

def loss(w, X, y, sigma):
    return Psi_m(y * (X @ w), sigma)


def SSVM_objective(w, X, y, sigma, lbd):
    return 1 / 2 * np.linalg.norm(w)**2 * lbd + loss(w, X, y, sigma)


def SVM_objective(w, X, y, C):
    return 1 / 2 * np.linalg.norm(w)**2 + C * max(0, 1 - y * (X @ w))

In [5]:
for i, n in enumerate(experiment_sizes):
    n = int(n)
    #X, _, y = sample_from_logreg(n=n, p=50)
    X, y = make_classification(n_samples=n, n_features=50, n_informative=25)

    #max_iter = int(n * np.sqrt(np.log(n)))
    max_iter = int(n * 1.25)
    print(f"running experiments for size {n} | max_iter = {max_iter}")
    c_val = 1/n
    clf = LinearSVC(loss='hinge', fit_intercept=False, C=c_val)
    true_start = time.time()
    true_params = []
    true_loss = []
    for j in range(n):
        X_temp = np.delete(X, (j), axis=0)
        y_temp = np.delete(y, (j), axis=0)
        clf.fit(X_temp, y_temp)
        true_params.append(clf.coef_[0])
        true_loss.append(SVM_objective(clf.coef_[0], X[j], y[j], c_val))
    true_end = time.time()
    print(f"\t true CV: {true_end - true_start}s")
    
    clf = SVM_smooth(sigma=sigma, lbd=lbd)
    approx_start = time.time()
    clf.fit(X, y, thresh=1e-3, n_iter=max_iter, sgd=True, batch_size=75, cv=False, approx_cv=True)
    approx_end = time.time()
    print(f"\t approx CV: {approx_end - approx_start}s")

    approx_loss = []
    for j in range(n):
        approx_loss.append(SSVM_objective(clf.approx_cv_obj.iterates[j], X[j], y[j], sigma, lbd))

    time_taken["true"][i] = true_end - true_start
    time_taken["IACV"][i] = approx_end - approx_start

    true_params_norm = normalize(np.asarray(true_params), axis=1)
    approx_params_norm = normalize(clf.approx_cv_obj.iterates, axis=1)
    err_approx.append(np.linalg.norm(true_params_norm - approx_params_norm, 2, axis=1).mean())
    
    true_loss_norm = normalize(np.asarray(true_loss).reshape(-1, 1), axis=0)
    approx_loss_norm = normalize(np.asarray(approx_loss).reshape(-1, 1), axis=0)
    err_cv.append(np.abs(true_loss_norm - approx_loss_norm).mean())
    print(f"\t err approx {err_approx[-1]} | err cv {err_cv[-1]}")

running experiments for size 1000 | max_iter = 1250
	 true CV: 1.5354785919189453s
	 approx CV: 2.896798849105835s
	 err approx 0.793336077643024 | err cv 0.00017410815310488865
running experiments for size 5750 | max_iter = 7187
	 true CV: 31.482812881469727s
	 approx CV: 11.934655904769897s
	 err approx 0.8707128254947899 | err cv 1.516133165333838e-05
running experiments for size 10500 | max_iter = 13125
	 true CV: 116.27933716773987s
	 approx CV: 22.820168495178223s
	 err approx 0.8585410577732099 | err cv 4.941643505980331e-06
running experiments for size 15250 | max_iter = 19062
	 true CV: 267.3279986381531s
	 approx CV: 33.64903235435486s
	 err approx 0.9846476251435953 | err cv 3.399466003645443e-06
running experiments for size 20000 | max_iter = 25000


KeyboardInterrupt: 

In [ ]:
fig, ax = plt.subplots()
ax.plot(experiment_sizes, time_taken["IACV"], label="IACV", c='black')
ax.plot(experiment_sizes, time_taken["true"], label="True", c='green')
ax.set_xlabel("Dataset size")
ax.set_ylabel("Time taken (s)")

ws = 1.45 # 1.75
hs = 1.35 # 1.25
fig.set_size_inches(w=5.73/ws, h=3.5/hs)
ax.legend()
plt.show()
plt.savefig('true_iacv_benchmark.pgf', bbox_inches='tight', pad_inches=0)

In [ ]:
fig, ax = plt.subplots()
ax.plot(experiment_sizes, err_approx, label="IACV", c='black')
ax.set_xlabel("Dataset size")
ax.set_ylabel("Err Approx")

ws = 1.45 # 1.75
hs = 1.35 # 1.25
fig.set_size_inches(w=5.73/ws, h=3.5/hs)
plt.show()
plt.savefig('iacv_err_approx_benchmark_classification.pgf', bbox_inches='tight', pad_inches=0)

In [ ]:
fig, ax = plt.subplots()
ax.plot(experiment_sizes, err_cv, label="IACV", c='black')
ax.set_xlabel("Dataset size")
ax.set_ylabel("Err CV")

ws = 1.45 # 1.75
hs = 1.35 # 1.25
fig.set_size_inches(w=5.73/ws, h=3.5/hs)
plt.show()
plt.savefig('iacv_err_cv_benchmark_classification.pgf', bbox_inches='tight', pad_inches=0)